# Video Games Dataset 

## This Notebook is for exploring capabilities of langchain --> pandas compatible agents
* YT video that provided pattern for using langchain agent: [[video]](https://www.youtube.com/watch?v=rFQ5Kmkd4jc) 
* Source file for the Games DF: [[source]](https://corgis-edu.github.io/corgis/csv/video_games/)

In [12]:
from Functions.General_Func import print_X, delete_variables

print_X("My functions imported")

==================== My functions imported ====================


In [13]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_pandas_dataframe_agent

print_X("General modules imported")


================== General modules imported ==================


## 1️⃣ Loading Games Dataset, .info(), .describe()

In [14]:
df_games = pd.read_csv('./Data/video_games.csv')
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Data columns (total 36 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Title                          1212 non-null   object 
 1   Features.Handheld?             1212 non-null   bool   
 2   Features.Max Players           1212 non-null   int64  
 3   Features.Multiplatform?        1212 non-null   bool   
 4   Features.Online?               1212 non-null   bool   
 5   Metadata.Genres                1212 non-null   object 
 6   Metadata.Licensed?             1212 non-null   bool   
 7   Metadata.Publishers            948 non-null    object 
 8   Metadata.Sequel?               1212 non-null   bool   
 9   Metrics.Review Score           1212 non-null   int64  
 10  Metrics.Sales                  1212 non-null   float64
 11  Metrics.Used Price             1212 non-null   float64
 12  Release.Console                1212 non-null   o

Leaving only required columns:

In [15]:
required_columns = [
    "Title",
    "Metadata.Genres",
    "Metadata.Publishers",
    "Metadata.Sequel?",
    "Metrics.Review Score",
    "Metrics.Sales",
    "Metrics.Used Price",
    "Release.Rating",
    "Release.Year"
    ]

df_games = df_games[required_columns]
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1212 entries, 0 to 1211
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Title                 1212 non-null   object 
 1   Metadata.Genres       1212 non-null   object 
 2   Metadata.Publishers   948 non-null    object 
 3   Metadata.Sequel?      1212 non-null   bool   
 4   Metrics.Review Score  1212 non-null   int64  
 5   Metrics.Sales         1212 non-null   float64
 6   Metrics.Used Price    1212 non-null   float64
 7   Release.Rating        1212 non-null   object 
 8   Release.Year          1212 non-null   int64  
dtypes: bool(1), float64(2), int64(2), object(4)
memory usage: 77.1+ KB


In [18]:
df_games[:5]

,Title,Metadata.Genres,Metadata.Publishers,Metadata.Sequel?,Metrics.Review Score,Metrics.Sales,Metrics.Used Price,Release.Rating,Release.Year
0,Super Mario 64 DS,Action,Nintendo,True,85,4.69,24.95,E,2004
1,Lumines: Puzzle Fusion,Strategy,Ubisoft,True,89,0.56,14.95,E,2004
2,WarioWare Touched!,"Action,Racing / Driving,Sports",Nintendo,True,81,0.54,22.95,E,2004
3,Hot Shots Golf: Open Tee,Sports,Sony,True,81,0.49,12.95,E,2004
4,Spider-Man 2,Action,Activision,True,61,0.45,14.95,E,2004


TODO: create data distorter + categorical data renamer, so that part of the dataset can be sent to ChGPT without commercial risks